<a href="https://colab.research.google.com/github/raphaelcarlosr/linkedin2firebase/blob/main/Parse_LinkedIn_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps

1. Load and extract Basic_LinkedInDataExport_xx-xx-xxxx.zip
2. Convert all csv files as json
3. Convert jsons files into json resume scheme
4. Install firebase-admin
5. Update resume into firebase firestore



In [23]:
# https://pypi.python.org/pypi/libarchive
# pip install --user firebase-admin
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
!pip install --user firebase-admin
# !pip install --upgrade firebase-admin

import libarchive.public
import pandas
import firebase_admin
import os
import glob
import zipfile
import json
import datetime

# print( pandas.__version__ ) 

clear = lambda: os.system('clear')
clear()

# Set your values
zipped_file = '/content/Complete_LinkedInDataExport_02-19-2021.zip'
credential_file = '' #path to credential json file
output_path = '/content/linkedin/'

# Extract linkedin data as csv
from zipfile import ZipFile
with ZipFile(zipped_file, 'r') as zipObj:
    zipObj.extractall(output_path)

# Convert all csv as json
csv_files = glob.glob(output_path + "*.csv")
for csv_file in csv_files:
  # print( csv_file )
  name = csv_file.split("/").pop()
  name = name.replace(".csv", "").replace(" ", "-")
  name = name.lower()
  # print( name )
  try:
    df = pandas.read_csv(csv_file, encoding="utf-8")
    # print( df.head(1) ) #View first 10 data rows
    df.to_json( csv_file.replace(".csv", ".json"), orient='records' )
  except FileNotFoundError as e:
    print(e)
  except Exception as e:  
    print("An exception occurred:")
    continue    
     

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_file

clear()

# Set credential
from firebase_admin import credentials
cred = credentials.Certificate(credential_file)

# Import the Firebase service
from firebase_admin import auth

# Initialize the default app
try:
  default_app = firebase_admin.get_app();
except:
  default_app = firebase_admin.initialize_app()

print(default_app.name)    # "[DEFAULT]"

# Setup db
from firebase_admin import firestore
db = firestore.client()
collection = db.collection(u'resume')


json_files = glob.glob(output_path + "*.json")
for json_file in json_files:
  name = json_file.split("/").pop()
  name = name.replace(".json", "").replace(" ", "-")
  name = name.lower()  
  print(name)
  

  with open(json_file) as json_content:
      data = json.load(json_content)
      doc = {
          u'updatedAt': datetime.datetime.now(),
          u'values': data
      }
      collection.document(name).set(doc, merge=True)


# Remove files
files = glob.glob(output_path + "*.csv") +  glob.glob(output_path + "*.json") +  glob.glob(output_path + 'Jobs/**/*.csv', recursive=True) +  glob.glob(output_path + 'Jobs/**/*.json', recursive=True)
# print (files)
for f in files:
    try:
        # f.unlink()
        os.remove(f)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))

os.rmdir(output_path + 'Jobs/Job Application Answers')
os.rmdir(output_path + 'Jobs')
os.rmdir(output_path)



Requirement already up-to-date: libarchive in /usr/local/lib/python3.6/dist-packages (0.4.7)
An exception occurred:
[DEFAULT]
shares
reactions
messages
ad_targeting
languages
phonenumbers
hashtag_follows
mobile-applications
teammates
ads-clicked
logins
projects
videos
education
savedjobalerts
registration
invitations
rich-media
searchqueries
certifications
email-addresses
recommendations-received
positions
skills
learning
inferences_about_you
causes-you-care-about
courses
contacts
profile
member_follows
company-follows
comments
recommendations-given
